In [ ]:
from skimage import io
import pandas as pd
import numpy as np
import os
import random

In [ ]:
def predict_category(image):
    # receives (M,N,3) image and returns predicted category
    print(np.shape(image),end='\t')
    return 1

In [ ]:
def predict_pixel(image):
    # receives (M,N,3) image and returns predicted pixel
    print(np.shape(image),end='\t')
    return 1

In [ ]:
def draw_layer(draw=100,folder='layers',home='problem2',resize=0.2,crop_x=60,crop_y=60,cooling_alpha=0.001):
    # create directory
    if not os.path.exists('./%s'%(folder)): os.mkdir('./%s'%(folder))
    if not os.path.exists('./%s/draw'%(folder)): os.mkdir('./%s/draw'%(folder))

    # read label
    layer='L%04d'%(draw)
    mpm_dir='./%s/MPM/Layer%03dto%03d/%s/'%(home,(draw-1)//50*50+1,(draw-1)//50*50+50,layer)
    pixel=[]
    for temp in range(len(os.listdir(mpm_dir))):
        name='%sframe_%04d.bmp'%(mpm_dir,temp+1)
        pixel.append(np.sum(io.imread(name)>150))
    threshold=np.array([10,np.quantile(pixel,.25),np.quantile(pixel,.75)])
    category=[np.sum(temp>threshold) for temp in pixel]

    # read cmd
    cmd=pd.read_csv('./%s/Command_Part1/XYPT_Part01_%s.csv'%(home,layer),header=None)
    left=cmd[0].min()
    right=cmd[0].max()
    down=cmd[1].min()
    up=cmd[1].max()
    mag=1000
    width=int((right-left)*mag*resize)+1
    height=int((up-down)*mag*resize)+1
    power=np.zeros([width,height])
    speed=np.zeros([width,height])
    temporal=np.zeros([width,height])

    current_order=0
    locations=[]
    loc_x=[]
    loc_y=[]
    first_row=cmd[0:1]
    history_x=[first_row[0][0]]*3
    history_y=[first_row[1][0]]*3
    speed_max=-10000
    speed_min=10000
    for line in cmd.iterrows():
        raw_x=line[1][0]
        raw_y=line[1][1]
        history_x.pop()
        history_x.insert(0,raw_x)
        history_y.pop()
        history_y.insert(0,raw_y)
        # calculate speed
        last_speed=((history_x[2]-history_x[0])*(history_x[2]-history_x[0])+(history_y[2]-history_y[0])*(history_y[2]-history_y[0]))**0.5/2e-5
        if current_order>0:
            speed[current_x,current_y]=last_speed
            speed_max=max(last_speed,speed_max)
            speed_min=min(last_speed,speed_min)
        current_order+=1
        current_x=int((raw_x-left)*mag*resize)
        current_y=int((raw_y-down)*mag*resize)
        power[current_x,current_y]=line[1][2]
        temporal[current_x,current_y]=current_order
        if line[1][3]==2:
            locations.append([current_x,current_y])
            loc_x.append(raw_x)
            loc_y.append(raw_y)
    power_min=np.min(power)
    power_max=np.max(power)
    
    # generate photos
    photo_id=0
    for temp in locations:
        if photo_id%100==0: print(photo_id,end='\t')
        center_x,center_y=temp
        power_crop=np.zeros([crop_x,crop_y])
        speed_crop=np.zeros([crop_x,crop_y])
        temporal_crop=np.zeros([crop_x,crop_y])
        for x in range(center_x-int(crop_x*0.5),center_x+int(crop_x*0.5)):
            for y in range(center_y-int(crop_y*0.5),center_y+int(crop_y*0.5)):
                # consider locations in the window with time stamp earlier than center
                if x<0 or x>=width: continue
                if y<0 or y>=height: continue
                if temporal[x,y]==0 or temporal[x,y]>temporal[center_x,center_y]: continue
                # power crop
                original_power=(power[x,y]-power_min)/(power_max-power_min)
                power_crop[x+int(crop_x*0.5)-center_x,y+int(crop_y*0.5)-center_y]=int(original_power*255)
                # speed crop
                original_speed=(speed[x,y]-speed_min)/(speed_max-speed_min)
                speed_crop[x+int(crop_x*0.5)-center_x,y+int(crop_y*0.5)-center_y]=int(original_speed*255)
                # temporal crop
                elapsed_time=temporal[center_x,center_y]-temporal[x,y]
                temporal_crop[x+int(crop_x*0.5)-center_x,y+int(crop_y*0.5)-center_y]=int(np.exp(-elapsed_time*cooling_alpha)*255)
        rgb_image=np.stack((power_crop.astype(np.uint8),speed_crop.astype(np.uint8),temporal_crop.astype(np.uint8)),axis=-1)
        io.imsave('./%s/draw/%d.png'%(folder,photo_id),rgb_image,check_contrast=False)
        photo_id+=1

    pixel_results=[]
    category_results=[]
    for photo in range(photo_id):
        img=io.imread('./%s/draw/%d.png'%(folder,photo))
        pixel_results.append(predict_pixel(img))
        # predict category
        # category_results.append(predict_category(img))
    # use quantiles to predict category
    threshold_results=np.array([10,np.quantile(pixel_results,.25),np.quantile(pixel_results,.75)])
    category_results=[np.sum(temp>threshold_results) for temp in pixel_results]

    pd.DataFrame({'x':loc_x,'y':loc_y,
                  'pixel_label':pixel,'category_label':category,
                  'pixel_predict':pixel_results,'category_predict':category_results}).to_csv('./%s/heatmap.csv'%(folder),index=0)

In [ ]:
draw_layer()